## Preambule

In [1]:
# Plotting
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import xarray as xr
from plotly.subplots import make_subplots

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_8464\47999591.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Input

In [2]:
rules = ["GF", "PC", "PCC", "AP", "GDR", "ECPC"]
path_data = "K:/ECEMF/T5.2/"

## Read data

In [4]:
xr_dataread = xr.open_dataset(path_data + "xr_dataread.nc")
xr_2030 = xr.open_dataset(path_data + "xr_alloc_2030.nc")
# xr_2035 = xr.open_dataset(path_data+"xr_alloc_2035.nc")
xr_2040 = xr.open_dataset(path_data + "xr_alloc_2040.nc")

In [5]:
df_eng = pd.read_csv(
    "X:/user/dekkerm/Data/ENGAGE/PolicyScenarios/engage-internal_snapshot_1695642798.csv"
)
df_eng = df_eng[df_eng.Variable == "Emissions|Kyoto Gases"]
df_eng = df_eng.reset_index(drop=True)
regions_df = np.array(df_eng.Region)
regions_df[regions_df == "Argentine Republic"] = "ARG"
regions_df[regions_df == "Canada"] = "CAN"
regions_df[regions_df == "Commonwealth of Australia"] = "AUS"
regions_df[regions_df == "Federative Republic of Brazil"] = "BRA"
regions_df[regions_df == "People's Repulic of China"] = "CHN"
regions_df[regions_df == "European Union (28 member countries)"] = "EU"
regions_df[regions_df == "Republic of India"] = "IND"
regions_df[regions_df == "Republic of Indonesia"] = "IDN"
regions_df[regions_df == "State of Japan"] = "JPN"
regions_df[regions_df == "Russian Federation"] = "RUS"
regions_df[regions_df == "Kingdom of Saudi Arabia"] = "SAU"
regions_df[regions_df == "Republic of South Africa"] = "ZAF"
regions_df[regions_df == "Republic of Korea (South Korea)"] = "KOR"
regions_df[regions_df == "United Mexican States"] = "MEX"
regions_df[regions_df == "Republic of Turkey"] = "TUR"
regions_df[regions_df == "United States of America"] = "USA"
regions_df[regions_df == "Viet Nam "] = "VNM"
df_eng.Region = regions_df
df = pd.read_excel(
    "X:/user/dekkerm/Data//UNFCCC_Parties_Groups_noeu.xlsx", sheet_name="Country groups"
)
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [6]:
curpol = "GP_CurPol_T45"
ndc = "GP_NDC2030_T45"
nz = "GP_Glasgow"
df_eng_ref = df_eng[["Model", "Scenario", "Region"] + list(df_eng.keys()[5:])]
df_eng_ref = df_eng_ref[df_eng_ref.Scenario.isin([curpol, ndc, nz])]
scen = np.array(df_eng_ref.Scenario)
scen[scen == ndc] = "NDC"
scen[scen == curpol] = "CurPol"
scen[scen == nz] = "NetZero"
reg = np.array(df_eng_ref.Region)
reg[reg == "World"] = "WORLD"
df_eng_ref["Scenario"] = scen
df_eng_ref["Region"] = reg
dummy = df_eng_ref.melt(
    id_vars=["Scenario", "Model", "Region"], var_name="Time", value_name="Value"
)
dummy["Time"] = np.array(dummy["Time"].astype(int))
dummy = dummy.set_index(["Scenario", "Model", "Region", "Time"])
xr_eng = xr.Dataset.from_dataframe(dummy)
xr_eng = xr_eng.reindex(Time=np.arange(1850, 2101))
xr_eng = xr_eng.interpolate_na(dim="Time", method="linear")

Cost optimal

In [7]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(["Emissions|Kyoto Gases", "Policy Cost|Consumption Loss"])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel(
    "X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx",
    sheet_name="meta_Ch3vetted_withclimate",
)
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6_meta["ModelScenario"] = modscens_meta
df_ar6_meta = df_ar6_meta[["ModelScenario", "Category", "Policy_category"]]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(["C1", "C2"])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6["ModelScenario"] = modscens
df_ar6 = df_ar6.drop(["Model", "Scenario", "Unit"], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(
    id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value"
)
dummy["Time"] = np.array(dummy["Time"].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r = xr.Dataset.from_dataframe(dummy)
xr_scen_r = xr_scen_r.reindex(Time=np.arange(1850, 2101))
xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

In [8]:
df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
df_ar6 = df_ar6[df_ar6.Variable.isin(["Emissions|Kyoto Gases", "Policy Cost|Consumption Loss"])]
df_ar6 = df_ar6.reset_index(drop=True)

df_ar6_meta = pd.read_excel(
    "X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx",
    sheet_name="meta_Ch3vetted_withclimate",
)
mods = np.array(df_ar6_meta.Model)
scens = np.array(df_ar6_meta.Scenario)
modscens_meta = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6_meta["ModelScenario"] = modscens_meta
df_ar6_meta = df_ar6_meta[["ModelScenario", "Category", "Policy_category"]]
df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(["C3"])]
df_ar6_meta = df_ar6_meta.reset_index(drop=True)
ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

mods = np.array(df_ar6.Model)
scens = np.array(df_ar6.Scenario)
modscens = np.array([mods[i] + "|" + scens[i] for i in range(len(scens))])
df_ar6["ModelScenario"] = modscens
df_ar6 = df_ar6.drop(["Model", "Scenario", "Unit"], axis=1)
df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

dummy = df_ar6.melt(
    id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value"
)
dummy["Time"] = np.array(dummy["Time"].astype(int))
dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
xr_scen_r2 = xr.Dataset.from_dataframe(dummy)
xr_scen_r2 = xr_scen_r2.reindex(Time=np.arange(1850, 2101))
xr_scen_r2 = xr_scen_r2.interpolate_na(dim="Time", method="linear")

## Plot

In [9]:
rulecolors = [
    "sienna",
    "goldenrod",
    "forestgreen",
    "tomato",
    "mediumvioletred",
    "steelblue",
    "silver",
]
rules = ["GF", "PC", "PCC", "ECPC", "AP", "GDR"]
standard_settings = {"TrajUnc": "Medium", "NegEmis": 0.5, "Convergence_year": 2050}

In [10]:
COUNTRIES = ["EU"]
year = 2040

In [11]:
for REG in COUNTRIES:
    xr_nld = xr.open_dataset(path_data + "Allocations/xr_alloc_" + REG + ".nc")
    fig = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=[
            "<b>Allocations in " + str(year) + "<br>(absolute)",
            "<b>Emission reductions<br>(" + str(year) + " compared to 1990)",
        ],
        horizontal_spacing=0.07,
        vertical_spacing=0.06,
    )
    # 2030 allocations
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                np.nan * xr_dataread.GHG_hist.sel(Region="NLD", Time=1990)
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        1,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                np.nan * xr_dataread.GHG_hist.sel(Region="NLD", Time=1990)
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        1,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC", "NDC"],
            y=[
                np.nan * xr_dataread.GHG_hist.sel(Region="NLD", Time=1990),
                np.nan * xr_dataread.GHG_hist.sel(Region="NLD", Time=1990),
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min(),
            # xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines",
            marker={"size": 11, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        1,
    )
    try:
        fig.add_traces(
            go.Scatter(
                x=["Current<br>Policies"] * len(xr_eng.Model),
                y=xr_eng.sel(Time=year, Region=REG, Scenario="CurPol").Value,
                name="Current policies",
                mode="lines+markers",
                marker_color="silver",
                marker_line_color="black",
                marker_line_width=1,
                marker={"size": 9, "symbol": "square"},
                showlegend=False,
            ),
            1,
            1,
        )
        vals = np.array(
            xr_scen_r.sel(Time=year, Region=REG, Variable="Emissions|Kyoto Gases").Value
        )
        fig.add_traces(
            go.Violin(
                x=["Cost<br>optimal"] * len(vals),
                y=vals,
                showlegend=False,
                points=False,
                opacity=1.0,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            1,
        )
        vals2 = np.array(
            xr_scen_r2.sel(Time=year, Region=REG, Variable="Emissions|Kyoto Gases").Value
        )
        fig.add_traces(
            go.Violin(
                x=["Cost<br>optimal"] * len(vals2),
                y=vals2,
                showlegend=False,
                points=False,
                opacity=0.5,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            1,
        )
        med = np.nanmedian(vals)
        fig.add_traces(
            go.Scatter(
                x=["NDC", "Current<br>Policies", "Cost<br>optimal"],
                y=[np.nanmedian(vals)] * 3,
                line={"color": "grey", "width": 2, "dash": "dot"},
                mode="lines",
                showlegend=False,
            ),
            1,
            1,
        )
        fig.add_traces(
            go.Scatter(
                x=["NDC"],
                y=[np.nanmedian(vals)],
                marker={"color": "grey", "size": 15},
                mode="markers",
                showlegend=False,
            ),
            1,
            1,
        )
        for y in [1990, 2021]:
            regval = float(xr_dataread.sel(Region=REG, Time=y).GHG_hist)
            fig.add_traces(
                go.Scatter(
                    x=["NDC"],
                    y=[regval],
                    line={"color": "grey", "width": 0.75},
                    mode="markers",
                    showlegend=False,
                ),
                1,
                1,
            )
            fig.add_traces(
                go.Scatter(
                    x=["NDC"],
                    y=[regval],
                    text=str(y) + "   ",
                    line={"color": "grey", "width": 0.5},
                    textfont={"color": "grey", "size": 13},
                    textposition="middle left",
                    mode="text",
                    showlegend=False,
                ),
                1,
                1,
            )
    except:
        3
    # fig.add_traces(go.Scatter(x=['NDC', 'CurPol'],
    #                         y=[np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median())]*2,
    #                         line={'color': 'silver', 'width': 2, 'dash': 'dot'},
    #                         mode='lines',
    #                         showlegend=False),
    #                 1, 1
    #                 )
    # fig.add_traces(go.Scatter(x=['NDC'],
    #                         y=[np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median())],
    #                         marker={'color': "grey", 'size': 15},
    #                         mode='markers',
    #                         showlegend=False), 1, 1
    #                 )
    for rule_i, rule in enumerate(rules):
        for T_i, T in enumerate([1.5, 2.0]):
            vals = np.array(
                xr_nld.sel(Temperature=T, Risk=[0.5, 0.33][T_i], TrajUnc="Medium")[rule].sel(
                    Time=year
                )
            ).flatten()
            vals[vals == 0] = np.nan
            val_med = np.array(
                xr_nld.sel(
                    Temperature=T,
                    Risk=[0.5, 0.33][T_i],
                    TrajUnc="Medium",
                    NegEmis=0.5,
                    Convergence_year=2050,
                    Scenario="SSP2",
                )[rule].sel(Time=year)
            ).flatten()
            val_med[val_med == 0] = np.nan
            if T == 1.5:
                med = np.nanmedian(vals)
                fig.add_traces(
                    go.Scatter(
                        x=["NDC", "Current<br>Policies", "Cost<br>optimal"]
                        + list(rules[: rule_i + 1]),
                        y=[np.nanmedian(val_med)] * (len(rules[: rule_i + 1]) + 3),
                        line={"color": rulecolors[rule_i], "width": 1, "dash": "dot"},
                        mode="lines",
                        showlegend=False,
                    ),
                    1,
                    1,
                )
                fig.add_traces(
                    go.Scatter(
                        x=["NDC"],
                        y=[np.nanmedian(val_med)],
                        marker={"color": rulecolors[rule_i], "size": 15},
                        mode="markers",
                        showlegend=False,
                    ),
                    1,
                    1,
                )
            fig.add_traces(
                go.Violin(
                    x=[rule] * len(vals),
                    y=vals,
                    showlegend=False,
                    points=False,
                    opacity=1.0 - T_i * 0.5,
                    line={"color": "black", "width": 3},
                    line_color=rulecolors[rule_i],
                ),
                1,
                1,
            )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                np.nan * xr_dataread.GHG_hist.sel(Region="NLD", Time=1990)
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        1,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                np.nan * xr_dataread.GHG_hist.sel(Region="NLD", Time=1990)
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        1,
    )

    # PANEL B
    regval = float(xr_dataread.sel(Region=REG, Time=1990).GHG_hist)
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                np.nan
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        2,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                np.nan
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        2,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC", "NDC"],
            y=[
                np.nan,
                np.nan,
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100,
            # (xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines",
            marker={"size": 11, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        2,
    )
    try:
        fig.add_traces(
            go.Scatter(
                x=["Current<br>Policies"] * len(xr_eng.Model),
                y=(np.array(xr_eng.sel(Time=year, Region=REG, Scenario="CurPol").Value) - regval)
                / regval
                * -100,
                name="Current policies",
                mode="lines+markers",
                marker_color="silver",
                marker_line_color="black",
                marker_line_width=1,
                marker={"size": 9, "symbol": "square"},
                showlegend=False,
            ),
            1,
            2,
        )
        vals = (
            (
                np.array(
                    xr_scen_r.sel(Time=year, Region=REG, Variable="Emissions|Kyoto Gases").Value
                )
                - regval
            )
            / regval
            * -100
        )
        fig.add_traces(
            go.Violin(
                x=["Cost<br>optimal"] * len(vals),
                y=vals,
                showlegend=False,
                points=False,
                opacity=1.0,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            2,
        )
        vals2 = (
            (
                np.array(
                    xr_scen_r2.sel(Time=year, Region=REG, Variable="Emissions|Kyoto Gases").Value
                )
                - regval
            )
            / regval
            * -100
        )
        fig.add_traces(
            go.Violin(
                x=["Cost<br>optimal"] * len(vals2),
                y=vals2,
                showlegend=False,
                points=False,
                opacity=0.5,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            2,
        )
        med = np.nanmedian(vals)
        fig.add_traces(
            go.Scatter(
                x=["NDC", "Current<br>Policies", "Cost<br>optimal"],
                y=[np.nanmedian(vals)] * 3,
                line={"color": "grey", "width": 1, "dash": "dot"},
                mode="lines",
                showlegend=False,
            ),
            1,
            2,
        )
        fig.add_traces(
            go.Scatter(
                x=["NDC"],
                y=[np.nanmedian(vals)],
                marker={"color": "grey", "size": 15},
                mode="markers",
                showlegend=False,
            ),
            1,
            2,
        )
        # fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies']+list(rules[:rule_i+1]),
        #                         y=[0]*(len(rules[:rule_i+1])+2),
        #                         line={'color': 'black', 'width': 2, 'dash': 'dash'},
        #                         mode='lines',
        #                         showlegend=False), 1, 2
        #         )
    except:
        3
    for rule_i, rule in enumerate(rules):
        for T_i, T in enumerate([1.5, 2.0]):
            vals = (
                (
                    np.array(
                        xr_nld.sel(Temperature=T, Risk=[0.5, 0.33][T_i])[rule].sel(Time=year)
                    ).flatten()
                    - regval
                )
                / regval
                * -100
            )
            vals[vals == 0] = np.nan
            val_med = (
                (
                    np.array(
                        xr_nld.sel(
                            Temperature=T,
                            Risk=[0.5, 0.33][T_i],
                            TrajUnc="Medium",
                            NegEmis=0.5,
                            Convergence_year=2050,
                            Scenario="SSP2",
                        )[rule].sel(Time=year)
                    ).flatten()
                    - regval
                )
                / regval
                * -100
            )
            val_med[val_med == 0] = np.nan
            if T == 1.5:
                med = np.nanmedian(vals)
                fig.add_traces(
                    go.Scatter(
                        x=["NDC", "Current<br>Policies", "Cost<br>optimal"]
                        + list(rules[: rule_i + 1]),
                        y=[np.nanmedian(val_med)] * (len(rules[: rule_i + 1]) + 3),
                        line={"color": rulecolors[rule_i], "width": 1, "dash": "dot"},
                        mode="lines",
                        showlegend=False,
                    ),
                    1,
                    2,
                )
                fig.add_traces(
                    go.Scatter(
                        x=["NDC"],
                        y=[np.nanmedian(val_med)],
                        marker={"color": rulecolors[rule_i], "size": 15},
                        mode="markers",
                        showlegend=False,
                    ),
                    1,
                    2,
                )
                fig.add_traces(
                    go.Scatter(
                        x=["NDC"],
                        y=[np.nanmedian(val_med)],
                        text=str(1 + int(np.round(np.nanmedian(val_med), 0))) + "%  ",
                        line={"color": rulecolors[rule_i], "width": 0.5},
                        textfont={"color": rulecolors[rule_i], "size": 16},
                        textposition="middle left",
                        mode="text",
                        showlegend=False,
                    ),
                    1,
                    2,
                )
            fig.add_traces(
                go.Violin(
                    x=[rule] * len(vals),
                    y=vals,
                    showlegend=False,
                    points=False,
                    opacity=1.0 - T_i * 0.5,
                    line={"color": "black", "width": 3},
                    line_color=rulecolors[rule_i],
                ),
                1,
                2,
            )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                np.nan
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        2,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                np.nan
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        2,
    )
    fig.update_annotations(font=dict(family="Helvetica", size=25))
    fig.update_layout(
        height=800, width=1900, template="simple_white", font=dict(family="Helvetica")
    )
    fig.update_yaxes(
        tickfont=dict(size=25),
        row=1,
        col=1,
        title="GHG emissions allocations in "
        + str(year)
        + " ("
        + REG
        + ")<br>[Mt CO<sub>2</sub>e]",
        titlefont=(dict(size=20)),
    )  # , range=(0, 250))
    fig.update_xaxes(tickfont=dict(size=20), row=1, col=1)
    fig.update_yaxes(
        tickfont=dict(size=25),
        row=1,
        col=2,
        title="% reduction with respect to 1990",
        titlefont=(dict(size=20)),
        range=(0, 150),
        tickvals=[0, 50, 100, 150, 200],
        ticktext=["0%", "50%", "100%", "150%", "200%"],
    )
    fig.update_xaxes(tickfont=dict(size=20), row=1, col=2)
    fig.show()
    fig.write_image(
        "X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/Alloc"
        + str(year)
        + "_"
        + REG
        + ".png",
        scale=3,
    )

KeyError: "'TrajUnc' is not a valid dimension or coordinate for Dataset with dimensions FrozenMappingWarningOnValuesAccess({'NegEmis': 7, 'NonCO2red': 5, 'Temperature': 10, 'Risk': 5, 'Timing': 2, 'Time': 80, 'Scenario': 5, 'Convergence_year': 13, 'Discount_factor': 4, 'Historical_startyear': 3, 'Capability_threshold': 3, 'RCI_weight': 3})"

Major emitters

In [ ]:
year = 2030
fig = make_subplots(
    rows=1,
    cols=4,
    subplot_titles=["<b>United States", "<b>India", "<b>China", "<b>Brazil"],
    horizontal_spacing=0.02,
    vertical_spacing=0.06,
    shared_yaxes=True,
)
for REG_i, REG in enumerate(["USA", "IND", "CHN", "BRA"]):
    xr_nld = xr.open_dataset(path_data + "Allocations/xr_alloc_" + REG + ".nc")
    # PANEL B
    regval = float(xr_dataread.sel(Region=REG, Time=1990).GHG_hist)
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                np.nan
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        1 + REG_i,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                np.nan
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        1 + REG_i,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC", "NDC"],
            y=[
                np.nan,
                np.nan,
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100,
            # (xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines",
            marker={"size": 11, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        1 + REG_i,
    )
    try:
        fig.add_traces(
            go.Scatter(
                x=["CP"] * len(xr_eng.Model),
                y=(np.array(xr_eng.sel(Time=year, Region=REG, Scenario="CurPol").Value) - regval)
                / regval
                * -100,
                name="Current policies",
                mode="lines+markers",
                marker_color="silver",
                marker_line_color="black",
                marker_line_width=1,
                marker={"size": 9, "symbol": "square"},
                showlegend=False,
            ),
            1,
            1 + REG_i,
        )
        vals = (
            (
                np.array(
                    xr_scen_r.sel(Time=year, Region=REG, Variable="Emissions|Kyoto Gases").Value
                )
                - regval
            )
            / regval
            * -100
        )
        fig.add_traces(
            go.Violin(
                x=["CO"] * len(vals),
                y=vals,
                showlegend=False,
                points=False,
                opacity=1.0,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            1 + REG_i,
        )
        vals2 = (
            (
                np.array(
                    xr_scen_r2.sel(Time=year, Region=REG, Variable="Emissions|Kyoto Gases").Value
                )
                - regval
            )
            / regval
            * -100
        )
        fig.add_traces(
            go.Violin(
                x=["CO"] * len(vals2),
                y=vals2,
                showlegend=False,
                points=False,
                opacity=0.5,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            1 + REG_i,
        )
        med = np.nanmedian(vals)
        fig.add_traces(
            go.Scatter(
                x=["NDC", "CP", "CO"],
                y=[np.nanmedian(vals)] * 3,
                line={"color": "grey", "width": 1, "dash": "dot"},
                mode="lines",
                showlegend=False,
            ),
            1,
            1 + REG_i,
        )
        fig.add_traces(
            go.Scatter(
                x=["NDC"],
                y=[np.nanmedian(vals)],
                marker={"color": "grey", "size": 15},
                mode="markers",
                showlegend=False,
            ),
            1,
            1 + REG_i,
        )
    except:
        3
    for rule_i, rule in enumerate(rules):
        for T_i, T in enumerate([1.5, 2.0]):
            vals = (
                (
                    np.array(
                        xr_nld.sel(Temperature=T, Risk=[0.5, 0.33][T_i])[rule].sel(Time=year)
                    ).flatten()
                    - regval
                )
                / regval
                * -100
            )
            vals[vals == 0] = np.nan
            val_med = (
                (
                    np.array(
                        xr_nld.sel(
                            Temperature=T,
                            Risk=[0.5, 0.33][T_i],
                            TrajUnc="Medium",
                            NegEmis=0.5,
                            Convergence_year=2050,
                            Scenario="SSP2",
                        )[rule].sel(Time=year)
                    ).flatten()
                    - regval
                )
                / regval
                * -100
            )
            val_med[val_med == 0] = np.nan
            if T == 1.5:
                med = np.nanmedian(vals)
                fig.add_traces(
                    go.Scatter(
                        x=["NDC", "CP", "CO"] + list(rules[: rule_i + 1]),
                        y=[np.nanmedian(val_med)] * (len(rules[: rule_i + 1]) + 3),
                        line={"color": rulecolors[rule_i], "width": 1, "dash": "dot"},
                        mode="lines",
                        showlegend=False,
                    ),
                    1,
                    1 + REG_i,
                )
                fig.add_traces(
                    go.Scatter(
                        x=["NDC"],
                        y=[np.nanmedian(val_med)],
                        marker={"color": rulecolors[rule_i], "size": 15},
                        mode="markers",
                        showlegend=False,
                    ),
                    1,
                    1 + REG_i,
                )
                fig.add_traces(
                    go.Scatter(
                        x=["NDC"],
                        y=[np.nanmedian(val_med)],
                        # text=str(1+int(np.round(np.nanmedian(val_med),0)))+'%  ',
                        line={"color": rulecolors[rule_i], "width": 0.5},
                        # textfont={'color': rulecolors[rule_i], 'size': 16},
                        textposition="middle left",
                        mode="text",
                        showlegend=False,
                    ),
                    1,
                    1 + REG_i,
                )
            fig.add_traces(
                go.Violin(
                    x=[rule] * len(vals),
                    y=vals,
                    showlegend=False,
                    points=False,
                    opacity=1.0 - T_i * 0.5,
                    line={"color": "black", "width": 3},
                    line_color=rulecolors[rule_i],
                ),
                1,
                1 + REG_i,
            )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                np.nan
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        1 + REG_i,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                np.nan
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        1 + REG_i,
    )
    fig.update_annotations(font=dict(family="Helvetica", size=25))
    fig.update_layout(
        height=600, width=2000, template="simple_white", font=dict(family="Helvetica")
    )
    # fig.update_yaxes(tickfont=dict(size=25), row=1, col=1+REG_i, title='GHG emissions allocations in '+str(year)+' ('+REG+')<br>[Mt CO<sub>2</sub>e]', titlefont=(dict(size=20)))#, range=(0, 250))
    # fig.update_xaxes(tickfont=dict(size=20), row=1, col=1+REG_i)
    if REG_i == 0:
        fig.update_yaxes(
            tickfont=dict(size=25),
            row=1,
            col=1 + REG_i,
            title="% reduction with respect to 1990",
            titlefont=(dict(size=20)),
            range=(-400, 200),
            tickvals=[-400, -300, -200, -100, 0, 100, 200],
            ticktext=["-400%", "-300%", "-200%", "-100%", "0%", "100%", "200%"],
        )
    else:
        fig.update_yaxes(
            tickfont=dict(size=25),
            row=1,
            col=1 + REG_i,
            range=(0, 150),
            tickvals=[0, 50, 100, 150, 200],
            ticktext=["0%", "50%", "100%", "150%", "200%"],
        )
    fig.update_xaxes(tickfont=dict(size=17), row=1, col=1 + REG_i)
fig.show()
fig.write_image(
    "X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/MajorEmitters.png", scale=3
)

EU

In [ ]:
for REG in ["EU"]:
    xr_nld = xr.open_dataset(path_data + "Allocations/xr_alloc_" + REG + ".nc")
    fig = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=[
            "<b>Allocations in 2030<br>(absolute)",
            "<b>Emission reductions<br>(2030 compared to 1990)",
        ],
        horizontal_spacing=0.07,
        vertical_spacing=0.06,
    )
    # 2030 allocations
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                0.45 * xr_dataread.GHG_hist.sel(Region="EU", Time=1990)
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        1,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                0.39 * xr_dataread.GHG_hist.sel(Region="EU", Time=1990)
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        1,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC", "NDC"],
            y=[
                0.39 * xr_dataread.GHG_hist.sel(Region="EU", Time=1990),
                0.45 * xr_dataread.GHG_hist.sel(Region="EU", Time=1990),
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min(),
            # xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines",
            marker={"size": 11, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        1,
    )
    try:
        fig.add_traces(
            go.Scatter(
                x=["Current<br>Policies"] * len(xr_eng.Model),
                y=xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value,
                name="Current policies",
                mode="lines+markers",
                marker_color="silver",
                marker_line_color="black",
                marker_line_width=1,
                marker={"size": 9, "symbol": "square"},
                showlegend=False,
            ),
            1,
            1,
        )
        vals = np.array(
            xr_scen_r.sel(Time=2030, Region=REG, Variable="Emissions|Kyoto Gases").Value
        )
        fig.add_traces(
            go.Violin(
                x=["Cost<br>optimal"] * len(vals),
                y=vals,
                showlegend=False,
                points=False,
                opacity=1.0,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            1,
        )
        vals2 = np.array(
            xr_scen_r2.sel(Time=2030, Region=REG, Variable="Emissions|Kyoto Gases").Value
        )
        fig.add_traces(
            go.Violin(
                x=["Cost<br>optimal"] * len(vals2),
                y=vals2,
                showlegend=False,
                points=False,
                opacity=0.5,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            1,
        )
        med = np.nanmedian(vals)
        fig.add_traces(
            go.Scatter(
                x=["NDC", "Current<br>Policies", "Cost<br>optimal"],
                y=[np.nanmedian(vals)] * 3,
                line={"color": "grey", "width": 2, "dash": "dot"},
                mode="lines",
                showlegend=False,
            ),
            1,
            1,
        )
        fig.add_traces(
            go.Scatter(
                x=["NDC"],
                y=[np.nanmedian(vals)],
                marker={"color": "grey", "size": 15},
                mode="markers",
                showlegend=False,
            ),
            1,
            1,
        )
        for y in [1990, 2021]:
            regval = float(xr_dataread.sel(Region=REG, Time=y).GHG_hist)
            fig.add_traces(
                go.Scatter(
                    x=["NDC"],
                    y=[regval],
                    line={"color": "grey", "width": 0.75},
                    mode="markers",
                    showlegend=False,
                ),
                1,
                1,
            )
            fig.add_traces(
                go.Scatter(
                    x=["NDC"],
                    y=[regval],
                    text=str(y) + "   ",
                    line={"color": "grey", "width": 0.5},
                    textfont={"color": "grey", "size": 13},
                    textposition="middle left",
                    mode="text",
                    showlegend=False,
                ),
                1,
                1,
            )
    except:
        3
    # fig.add_traces(go.Scatter(x=['NDC', 'CurPol'],
    #                         y=[np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median())]*2,
    #                         line={'color': 'silver', 'width': 2, 'dash': 'dot'},
    #                         mode='lines',
    #                         showlegend=False),
    #                 1, 1
    #                 )
    # fig.add_traces(go.Scatter(x=['NDC'],
    #                         y=[np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median())],
    #                         marker={'color': "grey", 'size': 15},
    #                         mode='markers',
    #                         showlegend=False), 1, 1
    #                 )
    for rule_i, rule in enumerate(rules):
        for T_i, T in enumerate([1.5, 2.0]):
            vals = np.array(
                xr_nld.sel(Temperature=T, Risk=[0.5, 0.33][T_i])[rule].sel(Time=2030)
            ).flatten()
            vals[vals == 0] = np.nan
            vals_med = np.array(
                xr_nld.sel(
                    Temperature=T,
                    Risk=[0.5, 0.33][T_i],
                    TrajUnc="Medium",
                    NegEmis=0.5,
                    Convergence_year=2050,
                    Scenario="SSP2",
                )[rule].sel(Time=2030)
            ).flatten()
            vals_med[vals_med == 0] = np.nan
            if T == 1.5:
                med = np.nanmedian(vals)
                fig.add_traces(
                    go.Scatter(
                        x=["NDC", "Current<br>Policies", "Cost<br>optimal"]
                        + list(rules[: rule_i + 1]),
                        y=[np.nanmedian(vals_med)] * (len(rules[: rule_i + 1]) + 3),
                        line={"color": rulecolors[rule_i], "width": 1, "dash": "dot"},
                        mode="lines",
                        showlegend=False,
                    ),
                    1,
                    1,
                )
                fig.add_traces(
                    go.Scatter(
                        x=["NDC"],
                        y=[np.nanmedian(vals_med)],
                        marker={"color": rulecolors[rule_i], "size": 15},
                        mode="markers",
                        showlegend=False,
                    ),
                    1,
                    1,
                )
            fig.add_traces(
                go.Violin(
                    x=[rule] * len(vals),
                    y=vals,
                    showlegend=False,
                    points=False,
                    opacity=1.0 - T_i * 0.5,
                    line={"color": "black", "width": 3},
                    line_color=rulecolors[rule_i],
                ),
                1,
                1,
            )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                0.45 * xr_dataread.GHG_hist.sel(Region="EU", Time=1990)
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        1,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                0.39 * xr_dataread.GHG_hist.sel(Region="EU", Time=1990)
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        1,
    )

    # PANEL B
    regval = float(xr_dataread.sel(Region=REG, Time=1990).GHG_hist)
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                55
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        2,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                61
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        2,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC", "NDC"],
            y=[
                55,
                61,
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100,
            # (xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines",
            marker={"size": 11, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        2,
    )
    try:
        fig.add_traces(
            go.Scatter(
                x=["Current<br>Policies"] * len(xr_eng.Model),
                y=(np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value) - regval)
                / regval
                * -100,
                name="Current policies",
                mode="lines+markers",
                marker_color="silver",
                marker_line_color="black",
                marker_line_width=1,
                marker={"size": 9, "symbol": "square"},
                showlegend=False,
            ),
            1,
            2,
        )
        vals = (
            (
                np.array(
                    xr_scen_r.sel(Time=2030, Region=REG, Variable="Emissions|Kyoto Gases").Value
                )
                - regval
            )
            / regval
            * -100
        )
        fig.add_traces(
            go.Violin(
                x=["Cost<br>optimal"] * len(vals),
                y=vals,
                showlegend=False,
                points=False,
                opacity=1.0,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            2,
        )
        vals2 = (
            (
                np.array(
                    xr_scen_r2.sel(Time=2030, Region=REG, Variable="Emissions|Kyoto Gases").Value
                )
                - regval
            )
            / regval
            * -100
        )
        fig.add_traces(
            go.Violin(
                x=["Cost<br>optimal"] * len(vals2),
                y=vals2,
                showlegend=False,
                points=False,
                opacity=0.5,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            2,
        )
        med = np.nanmedian(vals)
        fig.add_traces(
            go.Scatter(
                x=["NDC", "Current<br>Policies", "Cost<br>optimal"],
                y=[np.nanmedian(vals)] * 3,
                line={"color": "grey", "width": 1, "dash": "dot"},
                mode="lines",
                showlegend=False,
            ),
            1,
            2,
        )
        fig.add_traces(
            go.Scatter(
                x=["NDC"],
                y=[np.nanmedian(vals)],
                marker={"color": "grey", "size": 15},
                mode="markers",
                showlegend=False,
            ),
            1,
            2,
        )
        # fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies']+list(rules[:rule_i+1]),
        #                         y=[0]*(len(rules[:rule_i+1])+2),
        #                         line={'color': 'black', 'width': 2, 'dash': 'dash'},
        #                         mode='lines',
        #                         showlegend=False), 1, 2
        #         )
    except:
        3
    for rule_i, rule in enumerate(rules):
        for T_i, T in enumerate([1.5, 2.0]):
            vals = (
                (
                    np.array(
                        xr_nld.sel(Temperature=T, Risk=[0.5, 0.33][T_i])[rule].sel(Time=2030)
                    ).flatten()
                    - regval
                )
                / regval
                * -100
            )
            vals[vals == 0] = np.nan
            val_med = (
                (
                    np.array(
                        xr_nld.sel(
                            Temperature=T,
                            Risk=[0.5, 0.33][T_i],
                            TrajUnc="Medium",
                            NegEmis=0.5,
                            Convergence_year=2050,
                            Scenario="SSP2",
                        )[rule].sel(Time=2030)
                    ).flatten()
                    - regval
                )
                / regval
                * -100
            )
            val_med[val_med == 0] = np.nan
            if T == 1.5:
                med = np.nanmedian(vals)
                fig.add_traces(
                    go.Scatter(
                        x=["NDC", "Current<br>Policies", "Cost<br>optimal"]
                        + list(rules[: rule_i + 1]),
                        y=[np.nanmedian(val_med)] * (len(rules[: rule_i + 1]) + 3),
                        line={"color": rulecolors[rule_i], "width": 1, "dash": "dot"},
                        mode="lines",
                        showlegend=False,
                    ),
                    1,
                    2,
                )
                fig.add_traces(
                    go.Scatter(
                        x=["NDC"],
                        y=[np.nanmedian(val_med)],
                        marker={"color": rulecolors[rule_i], "size": 15},
                        mode="markers",
                        showlegend=False,
                    ),
                    1,
                    2,
                )
                fig.add_traces(
                    go.Scatter(
                        x=["NDC"],
                        y=[np.nanmedian(vals)],
                        text=str(int(np.round(np.nanmedian(val_med), 0))) + "%  ",
                        line={"color": rulecolors[rule_i], "width": 0.5},
                        textfont={"color": rulecolors[rule_i], "size": 16},
                        textposition="middle left",
                        mode="text",
                        showlegend=False,
                    ),
                    1,
                    2,
                )
            fig.add_traces(
                go.Violin(
                    x=[rule] * len(vals),
                    y=vals,
                    showlegend=False,
                    points=False,
                    opacity=1.0 - T_i * 0.5,
                    line={"color": "black", "width": 3},
                    line_color=rulecolors[rule_i],
                ),
                1,
                2,
            )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                55
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        2,
    )
    fig.add_traces(
        go.Scatter(
            x=["NDC"],
            y=[
                61
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100],
            name="NDCs",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        2,
    )
    fig.update_annotations(font=dict(family="Helvetica", size=25))
    fig.update_layout(
        height=800, width=1900, template="simple_white", font=dict(family="Helvetica")
    )
    fig.update_yaxes(
        tickfont=dict(size=25),
        row=1,
        col=1,
        title="GHG emissions allocations in 2030 (" + REG + ")<br>[Mt CO<sub>2</sub>e]",
        titlefont=(dict(size=20)),
    )
    fig.update_xaxes(tickfont=dict(size=20), row=1, col=1)
    fig.update_yaxes(
        tickfont=dict(size=25),
        row=1,
        col=2,
        title="% reduction with respect to 1990",
        titlefont=(dict(size=20)),
        range=(0, 150),
        tickvals=[0, 50, 100, 150, 200],
        ticktext=["0%", "50%", "100%", "150%", "200%"],
    )
    fig.update_xaxes(tickfont=dict(size=20), row=1, col=2)
    fig.write_image(
        "X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/Alloc2030_" + REG + ".png",
        scale=3,
    )
    fig.show()

EU 2040

In [ ]:
for REG in ["EU"]:
    xr_nld = xr.open_dataset(path_data + "Allocations/xr_alloc_" + REG + ".nc")
    fig = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=[
            "<b>Allocations in 2040<br>(absolute)",
            "<b>Emission reductions<br>(2040 compared to 1990)",
        ],
        horizontal_spacing=0.07,
        vertical_spacing=0.06,
    )
    # 2030 allocations
    fig.add_traces(
        go.Scatter(
            x=["ESA"],
            y=[
                0.10 * xr_dataread.GHG_hist.sel(Region="EU", Time=1990)
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()],
            name="ESA",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        1,
    )
    fig.add_traces(
        go.Scatter(
            x=["ESA"],
            y=[
                0.05 * xr_dataread.GHG_hist.sel(Region="EU", Time=1990)
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()],
            name="ESA",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        1,
    )
    fig.add_traces(
        go.Scatter(
            x=["ESA", "ESA"],
            y=[
                0.05 * xr_dataread.GHG_hist.sel(Region="EU", Time=1990),
                0.10 * xr_dataread.GHG_hist.sel(Region="EU", Time=1990),
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min(),
            # xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()],
            name="ESA",
            line={"color": "black", "width": 3},
            mode="lines",
            marker={"size": 11, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        1,
    )
    try:
        fig.add_traces(
            go.Scatter(
                x=["Current<br>Policies"] * len(xr_eng.Model),
                y=xr_eng.sel(Time=2040, Region=REG, Scenario="CurPol").Value,
                name="Current policies",
                mode="lines+markers",
                marker_color="silver",
                marker_line_color="black",
                marker_line_width=1,
                marker={"size": 9, "symbol": "square"},
                showlegend=False,
            ),
            1,
            1,
        )
        vals = np.array(
            xr_scen_r.sel(Time=2040, Region=REG, Variable="Emissions|Kyoto Gases").Value
        )
        fig.add_traces(
            go.Violin(
                x=["Cost<br>optimal"] * len(vals),
                y=vals,
                showlegend=False,
                points=False,
                opacity=1.0,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            1,
        )
        vals2 = np.array(
            xr_scen_r2.sel(Time=2040, Region=REG, Variable="Emissions|Kyoto Gases").Value
        )
        fig.add_traces(
            go.Violin(
                x=["Cost<br>optimal"] * len(vals2),
                y=vals2,
                showlegend=False,
                points=False,
                opacity=0.5,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            1,
        )
        med = np.nanmedian(vals)
        fig.add_traces(
            go.Scatter(
                x=["ESA", "Current<br>Policies", "Cost<br>optimal"],
                y=[np.nanmedian(vals)] * 3,
                line={"color": "grey", "width": 2, "dash": "dot"},
                mode="lines",
                showlegend=False,
            ),
            1,
            1,
        )
        fig.add_traces(
            go.Scatter(
                x=["ESA"],
                y=[np.nanmedian(vals)],
                marker={"color": "grey", "size": 15},
                mode="markers",
                showlegend=False,
            ),
            1,
            1,
        )
        for y in [1990, 2021]:
            regval = float(xr_dataread.sel(Region=REG, Time=y).GHG_hist)
            fig.add_traces(
                go.Scatter(
                    x=["ESA"],
                    y=[regval],
                    line={"color": "grey", "width": 0.75},
                    mode="markers",
                    showlegend=False,
                ),
                1,
                1,
            )
            fig.add_traces(
                go.Scatter(
                    x=["ESA"],
                    y=[regval],
                    text=str(y) + "   ",
                    line={"color": "grey", "width": 0.5},
                    textfont={"color": "grey", "size": 13},
                    textposition="middle left",
                    mode="text",
                    showlegend=False,
                ),
                1,
                1,
            )
    except:
        3
    # fig.add_traces(go.Scatter(x=['NDC', 'CurPol'],
    #                         y=[np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median())]*2,
    #                         line={'color': 'silver', 'width': 2, 'dash': 'dot'},
    #                         mode='lines',
    #                         showlegend=False),
    #                 1, 1
    #                 )
    # fig.add_traces(go.Scatter(x=['NDC'],
    #                         y=[np.array(xr_eng.sel(Time=2030, Region=REG, Scenario="CurPol").Value.median())],
    #                         marker={'color': "grey", 'size': 15},
    #                         mode='markers',
    #                         showlegend=False), 1, 1
    #                 )
    for rule_i, rule in enumerate(rules):
        for T_i, T in enumerate([1.5, 2.0]):
            vals = np.array(
                xr_nld.sel(Temperature=T, Risk=[0.5, 0.33][T_i])[rule].sel(Time=2040)
            ).flatten()
            vals[vals == 0] = np.nan
            val_med = np.array(
                xr_nld.sel(
                    Temperature=T,
                    Risk=[0.5, 0.33][T_i],
                    TrajUnc="Medium",
                    NegEmis=0.5,
                    Convergence_year=2050,
                    Scenario="SSP2",
                )[rule].sel(Time=2040)
            ).flatten()
            val_med[val_med == 0] = np.nan
            if T == 1.5:
                med = np.nanmedian(vals)
                fig.add_traces(
                    go.Scatter(
                        x=["ESA", "Current<br>Policies", "Cost<br>optimal"]
                        + list(rules[: rule_i + 1]),
                        y=[np.nanmedian(val_med)] * (len(rules[: rule_i + 1]) + 3),
                        line={"color": rulecolors[rule_i], "width": 1, "dash": "dot"},
                        mode="lines",
                        showlegend=False,
                    ),
                    1,
                    1,
                )
                fig.add_traces(
                    go.Scatter(
                        x=["ESA"],
                        y=[np.nanmedian(val_med)],
                        marker={"color": rulecolors[rule_i], "size": 15},
                        mode="markers",
                        showlegend=False,
                    ),
                    1,
                    1,
                )
            fig.add_traces(
                go.Violin(
                    x=[rule] * len(vals),
                    y=vals,
                    showlegend=False,
                    points=False,
                    opacity=1.0 - T_i * 0.5,
                    line={"color": "black", "width": 3},
                    line_color=rulecolors[rule_i],
                ),
                1,
                1,
            )
    fig.add_traces(
        go.Scatter(
            x=["ESA"],
            y=[
                0.10 * xr_dataread.GHG_hist.sel(Region="EU", Time=1990)
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()],
            name="ESA",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        1,
    )
    fig.add_traces(
        go.Scatter(
            x=["ESA"],
            y=[
                0.05 * xr_dataread.GHG_hist.sel(Region="EU", Time=1990)
            ],  # [xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()],
            name="ESA",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        1,
    )

    # PANEL B
    regval = float(xr_dataread.sel(Region=REG, Time=1990).GHG_hist)
    fig.add_traces(
        go.Scatter(
            x=["ESA"],
            y=[
                90
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
            name="ESA",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        2,
    )
    fig.add_traces(
        go.Scatter(
            x=["ESA"],
            y=[
                95
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100],
            name="ESA",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        2,
    )
    fig.add_traces(
        go.Scatter(
            x=["ESA", "ESA"],
            y=[
                90,
                95,
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100,
            # (xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
            name="ESA",
            line={"color": "black", "width": 3},
            mode="lines",
            marker={"size": 11, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        2,
    )
    try:
        fig.add_traces(
            go.Scatter(
                x=["Current<br>Policies"] * len(xr_eng.Model),
                y=(np.array(xr_eng.sel(Time=2040, Region=REG, Scenario="CurPol").Value) - regval)
                / regval
                * -100,
                name="Current policies",
                mode="lines+markers",
                marker_color="silver",
                marker_line_color="black",
                marker_line_width=1,
                marker={"size": 9, "symbol": "square"},
                showlegend=False,
            ),
            1,
            2,
        )
        vals = (
            (
                np.array(
                    xr_scen_r.sel(Time=2040, Region=REG, Variable="Emissions|Kyoto Gases").Value
                )
                - regval
            )
            / regval
            * -100
        )
        fig.add_traces(
            go.Violin(
                x=["Cost<br>optimal"] * len(vals),
                y=vals,
                showlegend=False,
                points=False,
                opacity=1.0,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            2,
        )
        vals2 = (
            (
                np.array(
                    xr_scen_r2.sel(Time=2040, Region=REG, Variable="Emissions|Kyoto Gases").Value
                )
                - regval
            )
            / regval
            * -100
        )
        fig.add_traces(
            go.Violin(
                x=["Cost<br>optimal"] * len(vals2),
                y=vals2,
                showlegend=False,
                points=False,
                opacity=0.5,
                line={"color": "grey", "width": 3},
                line_color="grey",
            ),
            1,
            2,
        )
        med = np.nanmedian(vals)
        fig.add_traces(
            go.Scatter(
                x=["ESA", "Current<br>Policies", "Cost<br>optimal"],
                y=[np.nanmedian(vals)] * 3,
                line={"color": "grey", "width": 1, "dash": "dot"},
                mode="lines",
                showlegend=False,
            ),
            1,
            2,
        )
        fig.add_traces(
            go.Scatter(
                x=["ESA"],
                y=[np.nanmedian(vals)],
                marker={"color": "grey", "size": 15},
                mode="markers",
                showlegend=False,
            ),
            1,
            2,
        )
        # fig.add_traces(go.Scatter(x=['NDC', 'Current<br>Policies']+list(rules[:rule_i+1]),
        #                         y=[0]*(len(rules[:rule_i+1])+2),
        #                         line={'color': 'black', 'width': 2, 'dash': 'dash'},
        #                         mode='lines',
        #                         showlegend=False), 1, 2
        #         )
    except:
        3
    for rule_i, rule in enumerate(rules):
        for T_i, T in enumerate([1.5, 2.0]):
            vals = (
                (
                    np.array(
                        xr_nld.sel(Temperature=T, Risk=[0.5, 0.33][T_i])[rule].sel(Time=2040)
                    ).flatten()
                    - regval
                )
                / regval
                * -100
            )
            vals[vals == 0] = np.nan
            val_med = (
                (
                    np.array(
                        xr_nld.sel(
                            Temperature=T,
                            Risk=[0.5, 0.33][T_i],
                            TrajUnc="Medium",
                            NegEmis=0.5,
                            Convergence_year=2050,
                            Scenario="SSP2",
                        )[rule].sel(Time=2040)
                    ).flatten()
                    - regval
                )
                / regval
                * -100
            )
            val_med[val_med == 0] = np.nan
            if T == 1.5:
                med = np.nanmedian(vals)
                fig.add_traces(
                    go.Scatter(
                        x=["ESA", "Current<br>Policies", "Cost<br>optimal"]
                        + list(rules[: rule_i + 1]),
                        y=[np.nanmedian(val_med)] * (len(rules[: rule_i + 1]) + 3),
                        line={"color": rulecolors[rule_i], "width": 1, "dash": "dot"},
                        mode="lines",
                        showlegend=False,
                    ),
                    1,
                    2,
                )
                fig.add_traces(
                    go.Scatter(
                        x=["ESA"],
                        y=[np.nanmedian(val_med)],
                        marker={"color": rulecolors[rule_i], "size": 15},
                        mode="markers",
                        showlegend=False,
                    ),
                    1,
                    2,
                )
                fig.add_traces(
                    go.Scatter(
                        x=["ESA"],
                        y=[np.nanmedian(val_med)],
                        text=str(int(np.round(np.nanmedian(val_med), 0))) + "%  ",
                        line={"color": rulecolors[rule_i], "width": 0.5},
                        textfont={"color": rulecolors[rule_i], "size": 16},
                        textposition="middle left",
                        mode="text",
                        showlegend=False,
                    ),
                    1,
                    2,
                )
            fig.add_traces(
                go.Violin(
                    x=[rule] * len(vals),
                    y=vals,
                    showlegend=False,
                    points=False,
                    opacity=1.0 - T_i * 0.5,
                    line={"color": "black", "width": 3},
                    line_color=rulecolors[rule_i],
                ),
                1,
                2,
            )
    fig.add_traces(
        go.Scatter(
            x=["ESA"],
            y=[
                90
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).max()-regval)/regval*-100],
            name="ESA",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-up"},
            showlegend=False,
        ),
        1,
        2,
    )
    fig.add_traces(
        go.Scatter(
            x=["ESA"],
            y=[
                95
            ],  # [(xr_dataread.GHG_ndc.sel(Region=REG).sel(Time=2030).min()-regval)/regval*-100],
            name="ESA",
            line={"color": "black", "width": 3},
            mode="lines+markers",
            marker={"size": 17, "symbol": "triangle-down"},
            showlegend=False,
        ),
        1,
        2,
    )
    fig.update_annotations(font=dict(family="Helvetica", size=25))
    fig.update_layout(
        height=800, width=1900, template="simple_white", font=dict(family="Helvetica")
    )
    fig.update_yaxes(
        tickfont=dict(size=25),
        row=1,
        col=1,
        title="GHG emissions allocations in 2040 (" + REG + ")<br>[Mt CO<sub>2</sub>e]",
        titlefont=(dict(size=20)),
    )
    fig.update_xaxes(tickfont=dict(size=20), row=1, col=1)
    fig.update_yaxes(
        tickfont=dict(size=25),
        row=1,
        col=2,
        title="% reduction with respect to 1990",
        titlefont=(dict(size=20)),
        range=(0, 150),
        tickvals=[0, 50, 100, 150, 200],
        ticktext=["0%", "50%", "100%", "150%", "200%"],
    )
    fig.update_xaxes(tickfont=dict(size=20), row=1, col=2)
    fig.write_image(
        "X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/Alloc2040_" + REG + ".png",
        scale=3,
    )
    fig.show()